In [8]:
import requests
import pandas as pd
from urllib.request import Request, urlopen
import json

In [10]:
with open("secret.json", "r") as f:
    config = json.load(f)

token = config["TOKEN_API_OD"]